# Generic Box
This code communicates with a component. The component was launched with the command below from the root of the repository:

```shell
$ docker run --rm -it -p 4061:8061 -v ./src/external1.py=/workspace/external.py generic
```


In [1]:
import matplotlib.pyplot as plt
import PIL.Image as PIL_image
import numpy as np

from importlib.machinery import SourceFileLoader
#In case of importing files from other directory
clip_pb2 = SourceFileLoader("clip_pb2","../protos/clip_pb2.py").load_module()
clip_pb2_grpc = SourceFileLoader("clip_pb2_grpc","../protos/clip_pb2_grpc.py").load_module()

import grpc
import io

## Set parameters (GRPC) and data
Set IP and port of the service (the same used in docker run), input data and call the service

In [2]:
from PIL import Image
import torch

#target='printart.isr.ist.utl.pt:8061'
target='localhost:8061'

texts = ["a diagram", "a dog", "a cat", "garden", "grass", "dirt road", "race car"]

file_paths = ["car.jpg","dog.jpg"]

image_byte_list = []
for path in file_paths:
    # Open the file in binary read mode ('rb') and read its entire content
    with open(path, 'rb') as f:
        image_bytes = f.read()
        image_byte_list.append(image_bytes)
        print(f"Read {path}: {len(image_bytes) / (1024 * 1024):.2f} MB")




request = clip_pb2.CLIPRequest(images= image_byte_list,
                               texts=texts) # list of strings

# List of file paths (example paths — replace with your actual ones)

channel_opt = [('grpc.max_send_message_length', 512 * 1024 * 1024), ('grpc.max_receive_message_length', 512 * 1024 * 1024)]
channel=grpc.insecure_channel(target,options=channel_opt)
estimator_stub = clip_pb2_grpc.CLIPServiceStub(channel)

Read car.jpg: 0.09 MB
Read dog.jpg: 0.37 MB


In [6]:
#Name of the method in the service
response = estimator_stub.Forward(request)
#After finishing channel.close


In [11]:
torch.load(io.BytesIO(response.similarity)).softmax(dim=-1)

tensor([[9.7670e-05, 1.8047e-04, 9.4234e-04, 1.3780e-05, 3.7842e-03, 2.6548e-03,
         9.9233e-01],
        [1.4549e-03, 3.1245e-01, 5.5151e-03, 8.3253e-03, 6.7070e-01, 1.9874e-04,
         1.3577e-03]])

In [13]:
torch.load(io.BytesIO(response.text_emb))

tensor([[ 0.0547, -0.0061,  0.0495,  ..., -0.6638, -0.1281, -0.4950],
        [ 0.1447,  0.0225, -0.2909,  ..., -0.4472, -0.3420,  0.1798],
        [ 0.1981, -0.2040, -0.1533,  ..., -0.4514, -0.5664,  0.0596],
        ...,
        [-0.0472, -0.4781,  0.0119,  ..., -0.1359, -0.1047,  0.3215],
        [ 0.0298, -0.2795, -0.0423,  ...,  0.4853, -0.2102,  0.0906],
        [ 0.1785, -0.3823, -0.1179,  ...,  0.2795, -0.1647, -0.1460]])